# Initialization

Test notebook for the DataHandlers. Test the CMAPSS DataHandler

First we import the necessary packages and create the global variables.

In [2]:
import numpy as np
import math
from data_handler_CMAPS import CMAPSDataHandler
from tunable_model import SequenceTunableModelRegression
from sklearn.preprocessing import StandardScaler, MinMaxScaler

from keras.models import Sequential, Model
from keras.layers import Dense, Input, Dropout, Reshape, Conv2D, Flatten, MaxPooling2D
from keras.optimizers import Adam
from keras.callbacks import LearningRateScheduler
from keras import backend as K
from keras import regularizers

import CMAPSAuxFunctions

Using TensorFlow backend.


# Define different types of Keras models

In [3]:
lambda_regularization = 0.20

def RNNModel(input_shape):
    """Define the RNN model"""
    
    #Create a sequential model
    model = Sequential()
    #model.add(Masking(mask_value=0, imput))
    model.add(LSTM(input_shape=input_shape))
    model.add(Dense(1, activation='linear', name='out'))
    
    return model
    

def RULmodel_SN_6(input_shape):
    #Create a sequential model
    model = Sequential()
    
    #Add the layers for the model
    model.add(Dense(10, input_dim=input_shape, activation='relu', kernel_initializer='glorot_normal', 
                    kernel_regularizer=regularizers.l2(lambda_regularization), name='fc1'))
    model.add(Dense(1, activation='linear', name='out'))
    
    return model

# Create the Data Handler

In [4]:
#Selected as per CNN paper
features = ['T2', 'T24', 'T30', 'T50', 'P2', 'P15', 'P30', 'Nf', 'Nc', 'epr', 'Ps30', 'phi', 'NRf', 'NRc', 
                     'BPR', 'farB', 'htBleed', 'Nf_dmd', 'PCNfR_dmd', 'W31', 'W32']
selected_indices = np.array([2, 3, 4, 7, 8, 9, 11, 12, 13, 14, 15, 17, 20, 21])
selected_features = list(features[i] for i in selected_indices-1)
data_folder = '../CMAPSSData'

window_size = 30
window_stride = 1
max_rul = 125

dHandler_cmaps = CMAPSDataHandler(data_folder, 1, selected_features, max_rul, window_size, window_stride)
#dHandler_cmaps.load_data(verbose=1, cross_validation_ratio=0.2)

# Create Tunable Model

In [5]:
#Shared parameters for the models
optimizer = Adam(lr=0, beta_1=0.5)
lossFunction = "mean_squared_error"
metrics = ["mse"]


#Create and compile the models
nFeatures = len(selected_features)
shapeSN = nFeatures*window_size
modelRULSN = RULmodel_SN_6(shapeSN)
modelRULSN.compile(optimizer = optimizer, loss = lossFunction, metrics = metrics)

#min_max_scaler = MinMaxScaler(feature_range=(-1, 1))
#min_max_scaler = None

tModel = SequenceTunableModelRegression('ModelRUL_SN_1', modelRULSN, lib_type='keras', data_handler=dHandler_cmaps)

# Load Data

In [13]:
min_max_scaler = MinMaxScaler(feature_range=(-1, 1))
tModel.data_scaler = min_max_scaler
#tModel.data_handler.change_dataset(2)
tModel.data_handler.sequence_length = 20
tModel.data_handler.sequence_stride = 3
tModel.data_handler.max_rul = 134

tModel.load_data(unroll=True, verbose=1, cross_validation_ratio=0)
tModel.print_data()

Loading data for dataset 2 with window_size of 20, stride of 3 and maxRUL of 134. Cros-Validation ratio 0
Loading data from memory without recomputing df
Printing shapes

Training data (X, y)
(16357, 280)
(16357,)
Testing data (X, y)
(259, 280)
(259,)
Printing first 5 elements

Training data (X, y)
[[-0.63844944 -0.37230904 -0.46733968 ...  0.32631579 -0.0212766
  -0.01454889]
 [-0.74595245 -0.39733362 -0.52129696 ... -0.41052632 -0.9752917
  -0.95211512]
 [-0.74169673 -0.43328598 -0.51726882 ... -0.41052632 -0.97117364
  -0.96191312]
 [-0.749283   -0.38438422 -0.51766666 ...  0.83157895  0.95813315
   0.95513427]
 [ 0.33018781  0.33870615  0.12380337 ...  0.38947368  0.26630062
   0.27886708]]
[129. 126. 123. 120. 117.]
Testing data (X, y)
[[ 0.34017948  0.33504535  0.15692369 ...  0.43157895  0.24227865
   0.26267759]
 [ 0.27560366  0.40826139  0.39766765 ...  0.89473684  0.96431023
   0.97507206]
 [ 0.28004441  0.38367392  0.37096253 ... -0.38947368 -0.68771448
  -0.66408805]
 [ 0.2

# Train the model and test some Tunable Model functionalities

In [6]:
lrate = LearningRateScheduler(CMAPSAuxFunctions.step_decay)
tModel.epochs = 100
tModel.train_model(learningRate_scheduler=lrate, verbose=1)

Epoch 1/100
17731/17731 [==============================] - 0s 26us/step - loss: 7906.7916 - mean_squared_error: 7903.5404
Epoch 2/100
17731/17731 [==============================] - 0s 9us/step - loss: 6999.5278 - mean_squared_error: 6994.9285
Epoch 3/100
17731/17731 [==============================] - 0s 11us/step - loss: 5899.5447 - mean_squared_error: 5892.1105
Epoch 4/100
17731/17731 [==============================] - 0s 9us/step - loss: 4751.5963 - mean_squared_error: 4740.1141
Epoch 5/100
17731/17731 [==============================] - 0s 8us/step - loss: 3710.5411 - mean_squared_error: 3694.3496
Epoch 6/100
17731/17731 [==============================] - 0s 8us/step - loss: 2885.1893 - mean_squared_error: 2864.1718
Epoch 7/100
17731/17731 [==============================] - 0s 10us/step - loss: 2308.2123 - mean_squared_error: 2282.9595
Epoch 8/100
17731/17731 [==============================] - 0s 10us/step - loss: 1937.7907 - mean_squared_error: 1909.3544
Epoch 9/100
17731/17731 [===

In [8]:
tModel.evaluate_model(['rhs', 'rmse'], round=2, setLimits=[7,145])
print("scores")

cScores = tModel.scores
rmse = math.sqrt(cScores['score_1'])
rmse2 = cScores['rmse']
rhs = cScores['rhs']
time = tModel.train_time
iterations = 10

i = range(1,len(tModel.y_test)+1)


for i, rul_prediction, rul_prediction_rounded, true_rul in zip(i, tModel.y_predicted, tModel.y_predicted_rounded, tModel.y_test):
    print('Engine {}, Predicted RUL {}, Rounded RUL {}, Real RUL {}'.format(i, rul_prediction, 
                                                                    rul_prediction_rounded, 
                                                                    true_rul))

print(cScores)
print("RMSE: {}".format(rmse))
print("RMSE2: {}".format(rmse2))
print("RHS: {}".format(rhs))
print("Time : {} seconds".format(time))

100/100 [==============================] - 0s 132us/step
scores
Engine 1, Predicted RUL [ 113.59394073], Rounded RUL 113.0, Real RUL 112.0
Engine 2, Predicted RUL [ 115.27581024], Rounded RUL 115.0, Real RUL 98.0
Engine 3, Predicted RUL [ 64.98041534], Rounded RUL 64.0, Real RUL 69.0
Engine 4, Predicted RUL [ 86.50939941], Rounded RUL 86.0, Real RUL 82.0
Engine 5, Predicted RUL [ 98.51219177], Rounded RUL 98.0, Real RUL 91.0
Engine 6, Predicted RUL [ 102.53627777], Rounded RUL 102.0, Real RUL 93.0
Engine 7, Predicted RUL [ 106.88496399], Rounded RUL 106.0, Real RUL 91.0
Engine 8, Predicted RUL [ 89.14979553], Rounded RUL 89.0, Real RUL 95.0
Engine 9, Predicted RUL [ 115.53969574], Rounded RUL 115.0, Real RUL 111.0
Engine 10, Predicted RUL [ 89.71224213], Rounded RUL 89.0, Real RUL 96.0
Engine 11, Predicted RUL [ 93.70278168], Rounded RUL 93.0, Real RUL 97.0
Engine 12, Predicted RUL [ 102.93409729], Rounded RUL 102.0, Real RUL 124.0
Engine 13, Predicted RUL [ 87.24149323], Rounded RUL 8